# Execute GPT requests with RAG

- TODO: Format different function calls in ./tools/templates
- TODO: Preprocess data from non-movie sets
- TODO: Make gpt call separate module, to swap out with local LLM
- TODO: Add better eval metrics

In [1]:
# ! pip install openai langchain -q

In [2]:
import sys
sys.path.append('tools')
from utils import *
import openai_chains

from sklearn.metrics import f1_score, recall_score, roc_auc_score

import os
import json
import csv

opai_api_key = "YOUR_API_KEY"

In [3]:
# Retrieve samples at random from formatted dataset
sample_size = 500
data_path = './../data/video/preprocessed_movies.json'

rows = extract_rows(sample_size, data_path)

In [4]:
# Print all reviews by this user (just for demo)
# first = list(rows[0].values())[0]

# for review in first:
#     print(f"{review['title']}")
#     print("*" * int(review['overall']))
#     print(f"Review: {review['reviewText']}")
#     print()

In [5]:
# prefs = format_preferences(rows[4])
# print(len(prefs['likes']))
# print(json.dumps(prefs, indent=4))

## Define model and function calling
**Only run one of the below cells**

In [6]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI

### Run inference and evaluate

In [11]:
model = ChatOpenAI(model_name="gpt-3.5-turbo", openai_api_key=opai_api_key, temperature=0)

chain = openai_chains.get_chain('keyphrases', model)

pred = []
truth = []
title = []
explanations = []

evaluated = 0

# Run inference on the chain
for i, user in enumerate(rows[5:]):    
    try:
        prefs = format_preferences(user)
    except Exception as e:
        print(f"Error: {e} for user {user}")
    pass
        
    # This is where the model is actually called
    response = chain.invoke(prefs)
    prediction = json.loads(response.additional_kwargs['function_call']['arguments'])
    
    print(prediction)

    
    # Log response
    # pred.append(prediction['keywords'])
    # truth.append(prefs['truth'])
    # title.append(prefs['target'])
    # explanations.append(response.additional_kwargs['function_call']['arguments'])
    
    selected_attributes = ['likes', 'dislikes', 'target', 'truth']

    print(json.dumps({attr: prefs[attr] for attr in selected_attributes}, indent=4))
    # if 'explanation' in prediction.keys():
    #     print(f"pred: {prediction['explanation']}")

    # print(f"pred: {prediction['reccomend']}, {prediction['explanation']}")
    evaluated += 1
    if evaluated == 2:
        break


{'keyphrases': ['prefers action/comedy movies', 'enjoys movies with strong female leads', 'likes movies with a mix of humor and action', 'appreciates movies with a twist', 'enjoys light-hearted and entertaining movies'], 'reccomend': True}
{
    "likes": [
        "Rocketeer VHS",
        "Hello, Dolly!",
        "Sherlock Holmes - A Game Of Shadows Edizione: Regno Unito  ITA italien",
        "Pitch Perfect",
        "White House Down",
        "Olympus has Fallen",
        "The Avengers (Avengers Assemble) Lenticular Steelbook Blu-ray 3D+2D Region Free Zavvi UK #/4000"
    ],
    "dislikes": [
        "Ride Along"
    ],
    "target": "Miss Congeniality / Miss Congeniality 2: Armed and Fabulous",
    "truth": true
}
{'keyphrases': ['prefers Japanese monster movies', 'enjoys live performances with dynamic sounds', "nostalgic for the 70's and 80's", 'likes goofy movies with a splash of sex', 'interested in ninja movies'], 'reccomend': True}
{
    "likes": [
        "Gamera",
        "B

## WIP

In [8]:
# f1 = f1_score(pred, truth)
# recall = recall_score(pred, truth)
# auc = roc_auc_score(pred, truth)

In [9]:
# print(f1, recall, auc)

In [10]:
# # Combine collected data into a list of tuples
# data = list(zip(pred, truth, title, explanations))

# # Define the file name for the CSV
# csv_filename = 'data_output.csv'

# # Save data to a CSV file
# with open(csv_filename, 'w', newline='') as csvfile:
#     csv_writer = csv.writer(csvfile)
    
#     # Write header
#     csv_writer.writerow(['Prediction', 'Truth', 'Title', 'Explanation'])
    
#     # Write rows
#     csv_writer.writerows(data)

# print(f"Data saved to {csv_filename}")